<a href="https://colab.research.google.com/github/sajidkhan2067/ML-Feature-Selection-PCA/blob/main/PCA_Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the required modules.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import ipaddress
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix

# Number_of_samples determine the highest number of samples(rows) from the attack and normal dataset should be read and used.

In [ ]:
number_of_samples = 100000
number_of_samples_attack =100000

In [ ]:
# First, we get the data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read data from attack and normal datasets located in google drive

In [ ]:
dfAttack = pd.concat(
    map(pd.read_csv, ['/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12udpfloodupf2.pcap_Flow.csv', '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12udpfloodupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpymasupf2.pcap_Flow.csv', '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpymasupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpxmasupf2.pcap_Flow.csv', '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpxmasupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpurgupf2.pcap_Flow.csv', '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpurgupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpsynupf2.pcap_Flow.csv', '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpsynupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcprstupf2.pcap_Flow.csv', '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcprstupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcppushupf2.pcap_Flow.csv', '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcppushupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpfinupf2.pcap_Flow.csv', '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpfinupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpackupf2.pcap_Flow.csv', '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Attack/12 May/12tcpackupf1.pcap_Flow.csv',
                       '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/26 may - sample/26maytcpsynupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/26 may - sample/26maytcpsynupf2.pcap_Flow.csv']),
    ignore_index=True)


default_value = 'attack'
dfAttack = dfAttack.assign(Label=default_value)



# Listed the features

In [ ]:
features=['Protocol','Flow Duration','Tot Fwd Pkts','Tot Bwd Pkts','TotLen Fwd Pkts','TotLen Bwd Pkts','Fwd Pkt Len Max','Fwd Pkt Len Min','Fwd Pkt Len Mean','Fwd Pkt Len Std','Bwd Pkt Len Max','Bwd Pkt Len Min','Bwd Pkt Len Mean','Bwd Pkt Len Std','Flow Byts/s','Flow Pkts/s','Flow IAT Mean','Flow IAT Std','Flow IAT Max','Flow IAT Min','Fwd IAT Tot','Fwd IAT Mean','Fwd IAT Std','Fwd IAT Max','Fwd IAT Min','Bwd IAT Tot','Bwd IAT Mean','Bwd IAT Std','Bwd IAT Max','Bwd IAT Min','Fwd PSH Flags','Bwd PSH Flags','Fwd URG Flags','Bwd URG Flags','Fwd Header Len','Bwd Header Len','Fwd Pkts/s','Bwd Pkts/s','Pkt Len Min','Pkt Len Max','Pkt Len Mean','Pkt Len Std','Pkt Len Var','FIN Flag Cnt','SYN Flag Cnt','RST Flag Cnt','PSH Flag Cnt','ACK Flag Cnt','URG Flag Cnt','CWE Flag Count','ECE Flag Cnt','Down/Up Ratio','Pkt Size Avg','Fwd Seg Size Avg','Bwd Seg Size Avg','Fwd Byts/b Avg','Fwd Pkts/b Avg','Fwd Blk Rate Avg','Bwd Byts/b Avg','Bwd Pkts/b Avg','Bwd Blk Rate Avg','Subflow Fwd Pkts','Subflow Fwd Byts','Subflow Bwd Pkts','Subflow Bwd Byts','Init Fwd Win Byts','Init Bwd Win Byts','Fwd Act Data Pkts','Fwd Seg Size Min','Active Mean','Active Std','Active Max','Active Min','Idle Mean','Idle Std','Idle Max','Idle Min','Label']
featuresWithoutLabel=['Protocol','Flow Duration','Tot Fwd Pkts','Tot Bwd Pkts','TotLen Fwd Pkts','TotLen Bwd Pkts','Fwd Pkt Len Max','Fwd Pkt Len Min','Fwd Pkt Len Mean','Fwd Pkt Len Std','Bwd Pkt Len Max','Bwd Pkt Len Min','Bwd Pkt Len Mean','Bwd Pkt Len Std','Flow Byts/s','Flow Pkts/s','Flow IAT Mean','Flow IAT Std','Flow IAT Max','Flow IAT Min','Fwd IAT Tot','Fwd IAT Mean','Fwd IAT Std','Fwd IAT Max','Fwd IAT Min','Bwd IAT Tot','Bwd IAT Mean','Bwd IAT Std','Bwd IAT Max','Bwd IAT Min','Fwd PSH Flags','Bwd PSH Flags','Fwd URG Flags','Bwd URG Flags','Fwd Header Len','Bwd Header Len','Fwd Pkts/s','Bwd Pkts/s','Pkt Len Min','Pkt Len Max','Pkt Len Mean','Pkt Len Std','Pkt Len Var','FIN Flag Cnt','SYN Flag Cnt','RST Flag Cnt','PSH Flag Cnt','ACK Flag Cnt','URG Flag Cnt','CWE Flag Count','ECE Flag Cnt','Down/Up Ratio','Pkt Size Avg','Fwd Seg Size Avg','Bwd Seg Size Avg','Fwd Byts/b Avg','Fwd Pkts/b Avg','Fwd Blk Rate Avg','Bwd Byts/b Avg','Bwd Pkts/b Avg','Bwd Blk Rate Avg','Subflow Fwd Pkts','Subflow Fwd Byts','Subflow Bwd Pkts','Subflow Bwd Byts','Init Fwd Win Byts','Init Bwd Win Byts','Fwd Act Data Pkts','Fwd Seg Size Min','Active Mean','Active Std','Active Max','Active Min','Idle Mean','Idle Std','Idle Max','Idle Min']

In [ ]:
dfAttack=dfAttack[features]

#Reading benign traffic from google drive

In [ ]:
dfBenign = pd.concat(
    map(pd.read_csv, ['/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/30upf2.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/30upf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/30pingupf2.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/30pingupf2.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/30pingupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/30Aprilupf2.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/30Aprilupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/30-1upf2.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/30-1upf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/29Aprilupf2.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/29Aprilupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/2mayupf2.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/2mayupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/2maysshupf2.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/2maysshupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/2maydownloadupf2.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/2maydownloadupf1.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/2maydownload-1upf2.pcap_Flow.csv',
                      '/content/drive/MyDrive/Colab Notebooks/LSTM/Behnam Dataset/2023/Main/Benign/2maydownload-1upf1.pcap_Flow.csv',
                      ]),
    ignore_index=True)


default_value = 'Benign'
dfBenign = dfBenign.assign(Label=default_value)


In [ ]:
print(features)
dfBenign=dfBenign[features]

['Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Av

In [ ]:
print(dfBenign.shape)
print(dfAttack.shape)

(967567, 78)
(5267322, 78)


#This code replaces infinite values (positive and negative) in the DataFrame dfAttack with NaN (Not a Number).

In [ ]:
dfAttack = dfAttack.replace([np.inf, -np.inf], np.nan)
dfBenign = dfBenign.replace([np.inf, -np.inf], np.nan)


# Replace NaN values in DataFrame with the mean of each column, making changes directly in the DataFrame

In [ ]:
dfAttack.fillna(dfAttack.mean(), inplace=True)
dfBenign.fillna(dfBenign.mean(), inplace=True)

<ipython-input-17-6ee1bfff7022>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  dfAttack.fillna(dfAttack.mean(), inplace=True)
<ipython-input-17-6ee1bfff7022>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  dfBenign.fillna(dfBenign.mean(), inplace=True)


In [ ]:
print(dfAttack.shape)
print(dfBenign.shape)

(5267322, 78)
(967567, 78)


In [ ]:
#Creating Y_normal using benign traffic
Y_normal = pd.concat([pd.DataFrame(dfBenign['Label'])], axis=0)
#Creating Y_attack using attack traffic
Y_attack= pd.concat([pd.DataFrame(dfAttack['Label'])], axis=0)
#Taking same number of attack and benign traffic
Y_attack=Y_attack.sample(n=Y_normal.shape[0])
#Creating X_normal using benign traffic
X_normal = pd.concat([pd.DataFrame(dfBenign[featuresWithoutLabel].values)], axis=0)
#Creating X_attack using attack traffic
X_attack = pd.concat([pd.DataFrame(dfAttack[featuresWithoutLabel].values)], axis=0)
#Ssame number of attack and benign traffic
X_attack=X_attack.sample(n=X_normal.shape[0])
#Combining benign and attack in X
X=np.concatenate((X_normal,X_attack))


print(X.shape)
print(X[0].shape)
print(len(X))
print(len(X_normal))
print(len(X_attack))
print(len(Y_normal))
print(len(Y_attack))
#Converting Pandas DataFrame to a one-dimensional NumPyarray
Y_attack=Y_attack.values.flatten()
Y_normal=Y_normal.values.flatten()
#Combining benign and attack in Y
Y=np.concatenate((Y_normal,Y_attack))

# Standardise the data

In [ ]:
scalar = StandardScaler(copy=True, with_mean=True, with_std=True)
scalar.fit(X)
X = scalar.transform(X)
print(X[0:1])
print(X.shape)

[[-0.28539563 -0.4090399  -0.01412572 -0.01747491 -0.00376818 -0.01603651
   0.55471482 -0.84568056 -0.75642858  0.9714108   1.8988987  -0.80106602
   1.1652471   2.43759471  0.01242649 -0.08146297 -0.9388192  -0.51416261
  -0.79373649 -0.73542277 -0.36473162 -0.75444583 -0.43272035 -0.69034726
  -0.60928681 -0.29158759 -0.24178009 -0.26771748 -0.3059247  -0.08886117
   0.         -0.25337275  0.         -0.21658901 -0.01159776 -0.01298653
  -0.07615525 -0.07824099 -0.80039696  1.84039318  0.29556707  1.85467792
   1.84129124 -0.3386021   1.5710584  -0.32310493 -0.25337275 -0.5939408
  -0.21658901 -0.2283747  -0.22917766  0.57676418  0.10972641 -0.75642858
   1.1652471   0.          0.          0.          0.          0.
   0.         -0.01412572 -0.00376818 -0.01747491 -0.01603651  0.
   1.07677657 -0.01113777  0.         -0.04038649 -0.08539932 -0.06398347
  -0.02679103 -0.74328368 -0.30229522 -0.67756583 -0.71095055]]
(1935134, 77)


# The class field, replace value 'attack' with 0 and 'normal' with 1

In [ ]:
for i in range(0,len(Y)):
  # print(Y[i])
  if Y[i] =="attack":
    Y[i]=0
  else:
    Y[i]=1



In [ ]:
#X is assigning to I because the value of X has changed several times before, so for the sake of easy calcuation it is done like that
I=X

In [ ]:
print(X[198:199])
print(X.shape)

[[-0.28539563 -0.40945496 -0.00957295 -0.01603179 -0.00378032 -0.0154674
   0.55471482 -0.84568056 -0.90321987  0.69862799  1.8988987  -0.80106602
   1.09908387  2.32741658  0.06890183 -0.07941868 -0.93956112 -0.51554831
  -0.79476133 -0.73542277 -0.36502654 -0.75541023 -0.43457532 -0.69156057
  -0.60928681 -0.29201446 -0.24242523 -0.26825663 -0.30634871 -0.0888622
   0.         -0.25337275  0.         -0.21658901 -0.00700946 -0.01149534
  -0.07400336 -0.07631391 -0.80039696  1.84039318  0.15426284  1.76476534
   1.67457049 -0.3386021   1.5710584  -0.32310493 -0.25337275 -0.5939408
  -0.21658901 -0.2283747  -0.22917766 -1.02978221 -0.04317892 -0.90321987
   1.09908387  0.          0.          0.          0.          0.
   0.         -0.00957295 -0.00378032 -0.01603179 -0.0154674   0.
   1.07677657 -0.01113777  0.         -0.04038649 -0.08539932 -0.06398347
  -0.02679103 -0.74328368 -0.30229522 -0.67756583 -0.71095055]]
(1935134, 77)


The line `X_train, X_test, Y_train, Y_test = train_test_split(I, Y, test_size=0.2, random_state=42)` is using the `train_test_split` function from a machine learning library, likely scikit-learn.

Here's a breakdown:

- `I`: This is the input data (features or independent variables).
- `Y`: This is the target variable (dependent variable or labels).

The function is splitting the dataset into training and testing sets:

- `X_train`: The training set of input data.
- `X_test`: The testing set of input data.
- `Y_train`: The training set of target variable.
- `Y_test`: The testing set of target variable.

- `test_size=0.2`: Specifies that 20% of the data will be used for testing, and the remaining 80% will be used for training.

- `random_state=42`: This provides a seed for the random number generator, ensuring that the split is reproducible. Using the same seed (in this case, 42) will result in the same split every time the code is run.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(I, Y, test_size = 0.2, random_state=42)

In [ ]:
print(X.shape)
print(I.shape)
print(X_train.shape)
print(X_test.shape)
print(" ")
print(Y_train.shape)
print(Y_test.shape)

(1935134, 77)
(1935134, 77)
(1548107, 77)
(387027, 77)
 
(1548107,)
(387027,)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
print(X[0:2])
print(X_train[0:2])

In [ ]:
#Represents the proportion of the dataset's variance explained by each principal component.
explained_variance = pca.explained_variance_ratio_

In [ ]:
print(explained_variance)

In [ ]:
# Create a DataFrame with the variances
variance_df = pd.DataFrame({'Feature': featuresWithoutLabel, 'Variance': explained_variance})

# Sort the variances in descending order
sorted_variances = variance_df.sort_values(by='Variance', ascending=False)

In [ ]:
# Print the sorted variances
print(sorted_variances[0:10])

            Feature  Variance
0          Protocol  0.184457
1     Flow Duration  0.141155
2      Tot Fwd Pkts  0.119178
3      Tot Bwd Pkts  0.095394
4   TotLen Fwd Pkts  0.046494
5   TotLen Bwd Pkts  0.036273
6   Fwd Pkt Len Max  0.035641
7   Fwd Pkt Len Min  0.033502
8  Fwd Pkt Len Mean  0.032982
9   Fwd Pkt Len Std  0.027852
